# Metropolis to Decode a Cipher

We obtain the coded message:

In [256]:
code = 'mprboydewyemo egfywhoptl gyed egybuyld yoel wyptgpxel gyrfyx tloemyretzyhbmpxp wyetyexx m oelpbtyldelydewyreuum gy xbtbkpwlwyetgyloeg owyldelyjpg tptqyqehydewyemeok gyldbw yjdbyjelxdyplyewyeywpqtemybuywlo wwyptyld yuptetxpemywfwl kybld owydes yhptt gyplybtyeyw op wybuyl xdtpxemyuexlbowywnxdyewyopwptqywdboll okyg rlywem wyrfyld ynwyqbs otk tlyetgyt jyxbohboel ylevyhbmpxp wybld oykeoz lwyldelyxetyr ylehh gyuboygbmmeowyptxmngptqyldobnqdyld ywjehwykeoz lyetgympinpgplfympt wykeptlept gyrfyqmbremyx tloemyretzwyeo tlyf lywdbjptqyeyrpqygbmmeoywin  c '
print(code)

mprboydewyemo egfywhoptl gyed egybuyld yoel wyptgpxel gyrfyx tloemyretzyhbmpxp wyetyexx m oelpbtyldelydewyreuum gy xbtbkpwlwyetgyloeg owyldelyjpg tptqyqehydewyemeok gyldbw yjdbyjelxdyplyewyeywpqtemybuywlo wwyptyld yuptetxpemywfwl kybld owydes yhptt gyplybtyeyw op wybuyl xdtpxemyuexlbowywnxdyewyopwptqywdboll okyg rlywem wyrfyld ynwyqbs otk tlyetgyt jyxbohboel ylevyhbmpxp wybld oykeoz lwyldelyxetyr ylehh gyuboygbmmeowyptxmngptqyldobnqdyld ywjehwykeoz lyetgympinpgplfympt wykeptlept gyrfyqmbremyx tloemyretzwyeo tlyf lywdbjptqyeyrpqygbmmeoywin  c 


**Goal: ** Decode this message.

** Assumption: ** Simple cipher which replaces each letter with some other letter. The possible letters are a-z and space.

## Coding and Decoding with Dictionaries

We will use Python dictionaries to represent coders and decoders. There are several other possible data structures one could use.

Python dictionaries are key / value pairs.

In [257]:
d = {'key1':'value1','key2':'value2'}

In [258]:
print(d)

{'key1': 'value1', 'key2': 'value2'}


In [259]:
d['key1']

'value1'

In [260]:
import string
import numpy as np
from numpy import random

We obtain all of the ascii lowercase letters plus the ' ' symbol

In [261]:
letters = list(string.ascii_lowercase)
letters = letters + [' ']
print(letters)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ']


In [262]:
letters_mix = letters.copy()
random.shuffle(letters_mix)
print(letters_mix)

['z', 'r', 'u', 'h', 'k', 'd', 'w', 'e', 'x', 'c', 'a', 's', 'o', 'v', 'f', 'm', 'i', 'b', 'j', 'q', ' ', 't', 'p', 'y', 'l', 'g', 'n']


In [263]:
cipher = dict(list(zip(letters,letters_mix)))
print(cipher['a']) ## a becomes
print(cipher['b']) ## b becomes

z
r


In [264]:
sample = "the yellow cat chased the brown dog"
sample_coded = "".join(list(map(lambda x: cipher[x], sample)))
print(sample_coded)
print(sample)

qeknlkssfpnuzqnuezjkhnqeknrbfpvnhfw
the yellow cat chased the brown dog


If we know the cipher, we can trivially invert it by reversing the role of keys and values in the dictionary.

In [265]:
decipher = dict(list(zip(letters_mix,letters)))

In [266]:
"".join(list(map(lambda x: decipher[x], sample_coded)))

'the yellow cat chased the brown dog'

Our goal is to find the deciper for the original code and use it to reconstruct the original message.

## Compute Probability of Data Given Cipher

We download 'War and Peace' and count the number of times we see $\gamma\beta$ where $\gamma$ and $\beta$ are any characters a-z and ' '. This requires a bit of data cleaning.

In [267]:
## download all of "War and Peace"
from urllib import request

In [268]:
data = request.urlopen("http://www.gutenberg.org/files/2600/2600-0.txt")

In [269]:
raw = data.read().decode('utf8')

In [270]:
raw[:5000]

'\ufeff\r\nThe Project Gutenberg EBook of War and Peace, by Leo Tolstoy\r\n\r\nThis eBook is for the use of anyone anywhere at no cost and with almost\r\nno restrictions whatsoever. You may copy it, give it away or re-use\r\nit under the terms of the Project Gutenberg License included with this\r\neBook or online at www.gutenberg.org\r\n\r\n\r\nTitle: War and Peace\r\n\r\nAuthor: Leo Tolstoy\r\n\r\nTranslators: Louise and Aylmer Maude\r\n\r\nPosting Date: January 10, 2009 [EBook #2600]\r\n\r\nLast Updated: December 17, 2016\r\n\r\nLanguage: English\r\n\r\nCharacter set encoding: UTF-8\r\n\r\n*** START OF THIS PROJECT GUTENBERG EBOOK WAR AND PEACE ***\r\n\r\n\r\n\r\n\r\nAn Anonymous Volunteer, and David Widger\r\n\r\n\r\n\r\n\r\n\r\n\r\nWAR AND PEACE\r\n\r\n\r\nBy Leo Tolstoy/Tolstoi\r\n\r\n\r\n\r\n\r\n\r\n    CONTENTS\r\n\r\n\r\n    BOOK ONE: 1805\r\n\r\n    CHAPTER I\r\n\r\n    CHAPTER II\r\n\r\n    CHAPTER III\r\n\r\n    CHAPTER IV\r\n\r\n    CHAPTER V\r\n\r\n    CHAPTER VI\r\n\r\n  

In [271]:
raw[10000:10500]

' seated himself on the sofa.\r\n\r\n“First of all, dear friend, tell me how you are. Set your friend’s\r\nmind at rest,” said he without altering his tone, beneath the\r\npoliteness and affected sympathy of which indifference and even irony\r\ncould be discerned.\r\n\r\n“Can one be well while suffering morally? Can one be calm in times\r\nlike these if one has any feeling?” said Anna Pávlovna. “You are\r\nstaying the whole evening, I hope?”\r\n\r\n“And the fete at the English ambassador’s? Today is Wednesday. I\r\nmust'

In [272]:
raw = raw.lower() ## make all lower case
raw = raw.replace("\r","")
raw = raw.replace("\n"," ") ## replace carriage return with space
cleaned = "".join(list(filter(lambda x: x in letters, raw))) ## remove all non letters

In [273]:
cleaned[:1000]

' the project gutenberg ebook of war and peace by leo tolstoy  this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever you may copy it give it away or reuse it under the terms of the project gutenberg license included with this ebook or online at wwwgutenbergorg   title war and peace  author leo tolstoy  translators louise and aylmer maude  posting date january   ebook   last updated december    language english  character set encoding utf   start of this project gutenberg ebook war and peace      an anonymous volunteer and david widger       war and peace   by leo tolstoytolstoi          contents       book one       chapter i      chapter ii      chapter iii      chapter iv      chapter v      chapter vi      chapter vii      chapter viii      chapter ix      chapter x      chapter xi      chapter xii      chapter xiii      chapter xiv      chapter xv      chapter xvi      chapter xvii      chapter xviii      chapter xix      chapter xx     

In [274]:
cleaned[10000:11000]

'we are ready to burn ours  prince vasli always spoke languidly like an actor repeating a stale part anna pvlovna schrer on the contrary despite her forty years overflowed with animation and impulsiveness to be an enthusiast had become her social vocation and sometimes even when she did not feel like it she became enthusiastic in order not to disappoint the expectations of those who knew her the subdued smile which though it did not suit her faded features always played round her lips expressed as in a spoiled child a continual consciousness of her charming defect which she neither wished nor could nor considered it necessary to correct  in the midst of a conversation on political matters anna pvlovna burst out  oh dont speak to me of austria perhaps i dont understand things but austria never has wished and does not wish for war she is betraying us russia alone must save europe our gracious sovereign recognizes his high vocation and will be true to it that is the one thing i have faith

Split `cleaned` into character pairs, there are `len(cleaned) -1` of these. 

In [275]:
n = len(cleaned)

In [276]:
fromix = list(range(0,n-1))

In [277]:
toix = list(range(2,n+1))

In [278]:
fromtoix = list(zip(fromix,toix))

In [279]:
fromtoix[:10]

[(0, 2),
 (1, 3),
 (2, 4),
 (3, 5),
 (4, 6),
 (5, 7),
 (6, 8),
 (7, 9),
 (8, 10),
 (9, 11)]

In [280]:
cleaned[fromtoix[0][0]:fromtoix[0][1]]

' t'

In [281]:
pairs = list(map(lambda x: cleaned[x[0]:x[1]], fromtoix))

In [282]:
pairs

[' t',
 'th',
 'he',
 'e ',
 ' p',
 'pr',
 'ro',
 'oj',
 'je',
 'ec',
 'ct',
 't ',
 ' g',
 'gu',
 'ut',
 'te',
 'en',
 'nb',
 'be',
 'er',
 'rg',
 'g ',
 ' e',
 'eb',
 'bo',
 'oo',
 'ok',
 'k ',
 ' o',
 'of',
 'f ',
 ' w',
 'wa',
 'ar',
 'r ',
 ' a',
 'an',
 'nd',
 'd ',
 ' p',
 'pe',
 'ea',
 'ac',
 'ce',
 'e ',
 ' b',
 'by',
 'y ',
 ' l',
 'le',
 'eo',
 'o ',
 ' t',
 'to',
 'ol',
 'ls',
 'st',
 'to',
 'oy',
 'y ',
 '  ',
 ' t',
 'th',
 'hi',
 'is',
 's ',
 ' e',
 'eb',
 'bo',
 'oo',
 'ok',
 'k ',
 ' i',
 'is',
 's ',
 ' f',
 'fo',
 'or',
 'r ',
 ' t',
 'th',
 'he',
 'e ',
 ' u',
 'us',
 'se',
 'e ',
 ' o',
 'of',
 'f ',
 ' a',
 'an',
 'ny',
 'yo',
 'on',
 'ne',
 'e ',
 ' a',
 'an',
 'ny',
 'yw',
 'wh',
 'he',
 'er',
 're',
 'e ',
 ' a',
 'at',
 't ',
 ' n',
 'no',
 'o ',
 ' c',
 'co',
 'os',
 'st',
 't ',
 ' a',
 'an',
 'nd',
 'd ',
 ' w',
 'wi',
 'it',
 'th',
 'h ',
 ' a',
 'al',
 'lm',
 'mo',
 'os',
 'st',
 't ',
 ' n',
 'no',
 'o ',
 ' r',
 're',
 'es',
 'st',
 'tr',
 'ri',
 'ic',

Count the number each pair. We use the `Counter` function in `collections`

In [283]:
from collections import Counter
z = ['blue', 'red', 'blue', 'yellow', 'blue', 'red']
counts = dict(Counter(z))
counts

{'blue': 3, 'red': 2, 'yellow': 1}

In [284]:
counts.keys()

dict_keys(['yellow', 'blue', 'red'])

In [285]:
counts.values()

dict_values([1, 3, 2])

In [286]:
counts = dict(Counter(pairs))

In [287]:
counts

{'  ': 17455,
 ' a': 69493,
 ' b': 25141,
 ' c': 21303,
 ' d': 17233,
 ' e': 11719,
 ' f': 21350,
 ' g': 9807,
 ' h': 49790,
 ' i': 30917,
 ' j': 1656,
 ' k': 4026,
 ' l': 13193,
 ' m': 19525,
 ' n': 15086,
 ' o': 34066,
 ' p': 18932,
 ' q': 1425,
 ' r': 15140,
 ' s': 40757,
 ' t': 87501,
 ' u': 5630,
 ' v': 3989,
 ' w': 40610,
 ' x': 450,
 ' y': 6735,
 ' z': 147,
 'a ': 15006,
 'aa': 25,
 'ab': 3459,
 'ac': 7018,
 'ad': 11265,
 'ae': 69,
 'af': 1671,
 'ag': 3379,
 'ah': 325,
 'ai': 8736,
 'aj': 168,
 'ak': 2534,
 'al': 14225,
 'am': 4602,
 'an': 45444,
 'ao': 42,
 'ap': 4632,
 'aq': 2,
 'ar': 17676,
 'as': 19823,
 'at': 28005,
 'au': 2512,
 'av': 4330,
 'aw': 2159,
 'ax': 51,
 'ay': 5206,
 'az': 353,
 'b ': 201,
 'ba': 3189,
 'bb': 229,
 'bc': 4,
 'bd': 20,
 'be': 11355,
 'bh': 2,
 'bi': 1006,
 'bj': 175,
 'bk': 2,
 'bl': 3638,
 'bm': 70,
 'bn': 22,
 'bo': 4088,
 'br': 1986,
 'bs': 483,
 'bt': 280,
 'bu': 5275,
 'bv': 47,
 'bw': 20,
 'by': 2563,
 'bz': 3,
 'c ': 610,
 'ca': 6722,
 'cc

There are `len(counts.keys())` pairs in War and Peace

In [288]:
len(counts.keys())

613

Total possible pairs is.

In [289]:
27*27

729

So some pairs are missing because they never appeared in the book. We add these to counts. This is important to avoid python errors in the Metropolis algorithm. By adding (artificial) 1 counts to these pairs, we make the algorithm a bit more robust.

In [290]:
## construct all possible pairs (NOTE: bad code)
allpairs = []
for i in letters:
    for j in letters:
        allpairs = allpairs + [i + j]

In [291]:
absent = list(set(allpairs) - set(counts.keys()))

In [292]:
absent = dict(list(zip(absent,len(absent)*[1])))

In [293]:
counts = {**counts,**absent} ## merge together dictionaries

In [294]:
counts

{'  ': 17455,
 ' a': 69493,
 ' b': 25141,
 ' c': 21303,
 ' d': 17233,
 ' e': 11719,
 ' f': 21350,
 ' g': 9807,
 ' h': 49790,
 ' i': 30917,
 ' j': 1656,
 ' k': 4026,
 ' l': 13193,
 ' m': 19525,
 ' n': 15086,
 ' o': 34066,
 ' p': 18932,
 ' q': 1425,
 ' r': 15140,
 ' s': 40757,
 ' t': 87501,
 ' u': 5630,
 ' v': 3989,
 ' w': 40610,
 ' x': 450,
 ' y': 6735,
 ' z': 147,
 'a ': 15006,
 'aa': 25,
 'ab': 3459,
 'ac': 7018,
 'ad': 11265,
 'ae': 69,
 'af': 1671,
 'ag': 3379,
 'ah': 325,
 'ai': 8736,
 'aj': 168,
 'ak': 2534,
 'al': 14225,
 'am': 4602,
 'an': 45444,
 'ao': 42,
 'ap': 4632,
 'aq': 2,
 'ar': 17676,
 'as': 19823,
 'at': 28005,
 'au': 2512,
 'av': 4330,
 'aw': 2159,
 'ax': 51,
 'ay': 5206,
 'az': 353,
 'b ': 201,
 'ba': 3189,
 'bb': 229,
 'bc': 4,
 'bd': 20,
 'be': 11355,
 'bf': 1,
 'bg': 1,
 'bh': 2,
 'bi': 1006,
 'bj': 175,
 'bk': 2,
 'bl': 3638,
 'bm': 70,
 'bn': 22,
 'bo': 4088,
 'bp': 1,
 'bq': 1,
 'br': 1986,
 'bs': 483,
 'bt': 280,
 'bu': 5275,
 'bv': 47,
 'bw': 20,
 'bx': 1,
 '

Transform the counts to conditional probabilities:

In [295]:
num_added = dict(Counter(list(map(lambda x: x[0], list(absent.keys())))))
num_added

{'b': 5,
 'c': 8,
 'd': 1,
 'f': 4,
 'g': 2,
 'h': 3,
 'i': 1,
 'j': 22,
 'k': 4,
 'l': 1,
 'm': 4,
 'p': 4,
 'q': 24,
 'r': 1,
 's': 1,
 't': 1,
 'v': 7,
 'w': 5,
 'x': 9,
 'y': 2,
 'z': 7}

In [296]:
counts_single = dict(Counter(cleaned))
for i in num_added:
    counts_single[i] = counts_single[i] + num_added[i]
counts_single

{' ': 583077,
 'a': 202717,
 'b': 34663,
 'c': 61630,
 'd': 118299,
 'e': 313575,
 'f': 54905,
 'g': 51329,
 'h': 167418,
 'i': 172258,
 'j': 2596,
 'k': 20436,
 'l': 96533,
 'm': 61653,
 'n': 184184,
 'o': 190083,
 'p': 45537,
 'q': 2355,
 'r': 148432,
 's': 162898,
 't': 226415,
 'u': 64399,
 'v': 27094,
 'w': 59214,
 'x': 4393,
 'y': 46237,
 'z': 2395}

In [297]:
for i in counts:
    counts[i] = counts[i] / counts_single[i[0]]

In [298]:
np.sum(np.array(list(counts.values())))

26.999998284960647

In [299]:
pairprob = counts

Normalize the counts of single letters.

In [300]:
n = np.sum(np.array(list(counts_single.values())))
n

3104725

In [301]:
for i in counts_single:
    counts_single[i] = counts_single[i] / n

In [302]:
singleprob = counts_single
singleprob

{' ': 0.187803106555331,
 'a': 0.06529306138224802,
 'b': 0.01116459589818744,
 'c': 0.01985038932594674,
 'd': 0.03810289156044416,
 'e': 0.10099928335037725,
 'f': 0.01768433597178494,
 'g': 0.016532543139891616,
 'h': 0.05392361642335473,
 'i': 0.05548253065891504,
 'j': 0.0008361449081641691,
 'k': 0.00658222547890715,
 'l': 0.031092286756476017,
 'm': 0.019857797389462833,
 'n': 0.059323772636868,
 'o': 0.06122377988388666,
 'p': 0.014666999492708693,
 'q': 0.0007585212861042443,
 'r': 0.04780842103567949,
 's': 0.052467770897583525,
 't': 0.07292594352156793,
 'u': 0.02074225575534065,
 'v': 0.008726698821956856,
 'w': 0.019072220567038948,
 'x': 0.00141494013157365,
 'y': 0.014892462295372376,
 'z': 0.0007714048748278833}

## Calculating the Probability of Strings that are Not Encoded

In [408]:
test = 'hello'

n = len(test)
toix = list(range(2,n+1))
fromix = list(range(0,n-1))
fromtoix = list(zip(fromix,toix))

In [409]:
pairs = list(map(lambda x: test[x[0]:x[1]], fromtoix))
pairs

['he', 'el', 'll', 'lo']

In [410]:
X = dict(Counter(pairs))
X

{'el': 1, 'he': 1, 'll': 1, 'lo': 1}

In [411]:
p = np.array(list(map(lambda y: pairprob[y], X.keys())))
p

array([0.4475385 , 0.13692727, 0.03273858, 0.08600168])

In [412]:
ns = np.array(list(X.values()))
ns

array([1, 1, 1, 1])

In [413]:
out = np.sum(ns*np.log(p)) + np.log(singleprob[test[0]])
print(out)
print(np.exp(out))

-11.585074416515086
9.303923021188655e-06


In [414]:
## if all len(test) character strings were equally likely
1.0 / np.power(27.0,len(test))

6.969171937625632e-08

## The Posterior

The posterior is proportional to the likelihood times the prior:
$$ \pi(\theta|x) \propto f(x|\theta)\pi(\theta) $$
Here $\theta$ represents the inverse cipher, the function which maps the code back to the true letter. This is a discrete parameter that could take 26! possible values. We put a uniform prior on all possible parameter values. Therefore the posterior is proportional to just the likelihood $f(x|\theta)$.

In python, we can represent the $\theta$ parameter with a dictionary where the values are the coded message and the key are what letter the code corresponds to. One could also use  a pandas data frame to accomplish this.

In [415]:
## random initial starting value for theta
theta = letters.copy()
random.shuffle(theta)
theta = dict(list(zip(letters,theta)))

In [416]:
theta

{' ': 'a',
 'a': 'e',
 'b': 'p',
 'c': 't',
 'd': 'f',
 'e': 'd',
 'f': 'g',
 'g': 'h',
 'h': 'q',
 'i': 'y',
 'j': 'i',
 'k': 'o',
 'l': 'k',
 'm': 'w',
 'n': 'z',
 'o': 'v',
 'p': 'j',
 'q': 'c',
 'r': 's',
 's': 'm',
 't': ' ',
 'u': 'x',
 'v': 'r',
 'w': 'n',
 'x': 'u',
 'y': 'b',
 'z': 'l'}

In [417]:
decoded = list(map(lambda x: theta[x], code))

In [418]:
decoded = "".join(decoded)

In [419]:
## decoded mesage does not appear to be correct, because theta was a random guess
decoded

'wjspvbfdnbdwvadhgbnqvj kahbdfadhbpxbkfabvdkanbj hjudkahbsgbua kvdwbsd lbqpwjujanbd bduuawavdkjp bkfdkbfdnbsdxxwahbaup pojnknbd hbkvdhavnbkfdkbijha j cbcdqbfdnbdwdvoahbkfpnabifpbidkufbjkbdnbdbnjc dwbpxbnkvannbj bkfabxj d ujdwbngnkaobpkfavnbfdmabqj  ahbjkbp bdbnavjanbpxbkauf judwbxdukpvnbnzufbdnbvjnj cbnfpvkkavobhaskbndwanbsgbkfabznbcpmav oa kbd hb aibupvqpvdkabkdrbqpwjujanbpkfavbodvlaknbkfdkbud bsabkdqqahbxpvbhpwwdvnbj uwzhj cbkfvpzcfbkfabnidqnbodvlakbd hbwjyzjhjkgbwj anbodj kdj ahbsgbcwpsdwbua kvdwbsd lnbdva kbgakbnfpij cbdbsjcbhpwwdvbnyzaata'

Compute the log likelihood (= log posterior because uniform prior) in pieces.

In [424]:
n = len(code)
toix = list(range(2,n+1))
fromix = list(range(0,n-1))
fromtoix = list(zip(fromix,toix))

In [425]:
pairs = list(map(lambda x: decoded[x[0]:x[1]], fromtoix))

In [426]:
X = dict(Counter(pairs))

In [427]:
X

{'  ': 1,
 ' a': 4,
 ' b': 5,
 ' c': 4,
 ' d': 2,
 ' h': 4,
 ' j': 2,
 ' k': 6,
 ' l': 2,
 ' o': 1,
 ' p': 1,
 ' u': 2,
 'a ': 5,
 'aa': 1,
 'ab': 8,
 'ad': 2,
 'ah': 7,
 'ai': 1,
 'ak': 3,
 'an': 7,
 'ao': 1,
 'as': 1,
 'at': 1,
 'au': 2,
 'av': 7,
 'aw': 1,
 'b ': 1,
 'ba': 1,
 'bc': 3,
 'bd': 14,
 'bf': 4,
 'bg': 1,
 'bh': 3,
 'bi': 3,
 'bj': 5,
 'bk': 13,
 'bn': 11,
 'bo': 3,
 'bp': 6,
 'bq': 3,
 'bs': 8,
 'bu': 4,
 'bv': 2,
 'bw': 2,
 'bx': 3,
 'bz': 1,
 'c ': 1,
 'cb': 5,
 'cd': 1,
 'cf': 1,
 'cp': 1,
 'cw': 1,
 'd ': 8,
 'db': 3,
 'df': 1,
 'dh': 3,
 'dj': 2,
 'dk': 8,
 'dm': 1,
 'dn': 5,
 'dq': 3,
 'dr': 1,
 'du': 2,
 'dv': 6,
 'dw': 9,
 'dx': 1,
 'f ': 1,
 'fa': 7,
 'fb': 3,
 'fd': 7,
 'fp': 4,
 'fv': 1,
 'ga': 1,
 'gb': 5,
 'gn': 1,
 'ha': 3,
 'hb': 11,
 'hg': 1,
 'hj': 3,
 'hp': 2,
 'ib': 1,
 'id': 2,
 'if': 1,
 'ij': 2,
 'j ': 13,
 'ja': 3,
 'jc': 2,
 'jd': 1,
 'jh': 2,
 'jk': 3,
 'jn': 2,
 'jp': 1,
 'js': 1,
 'ju': 4,
 'jy': 1,
 'ka': 7,
 'kb': 10,
 'kd': 3,
 'kf': 11,
 'k

In [428]:
p = np.array(list(map(lambda y: pairprob[y], X.keys())))
ns = np.array(list(X.values()))
out = np.sum(ns*np.log(p)) + np.log(singleprob[decoded[0]])

In [429]:
out

-3443.725205736844

We put the above code in functions which we can call in Metropolis.

In [433]:
def decode(theta):
    decoded = list(map(lambda x: theta[x], code))
    decoded = "".join(decoded)
    return decoded

In [434]:
def logpost(theta):
    decoded = decode(theta)
    pairs = list(map(lambda x: decoded[x[0]:x[1]], fromtoix))
    X = dict(Counter(pairs))
    p = np.array(list(map(lambda y: pairprob[y], X.keys())))
    ns = np.array(list(X.values()))
    return np.sum(ns*np.log(p)) + np.log(singleprob[decoded[0]])

In [436]:
## check that function output matches scratch work
logpost(theta)

-3443.725205736844

## Code Up Metropolis

The proposal distribution considers switching pairs of letters in the deciper theta.

In [437]:
def thetaproposal(theta):
    thetanew = theta.copy()
    ixs = random.choice(list(theta.keys()),size=2,replace=False)
    thetanew[ixs[0]] = theta[ixs[1]]
    thetanew[ixs[1]] = theta[ixs[0]]
    return thetanew


In [440]:
## random initial starting value for theta
theta = letters.copy()
random.shuffle(theta)
theta = dict(list(zip(letters,theta)))

In [441]:
## metropolis
niter = 10000
for ii in np.arange(niter):
    thetanew = thetaproposal(theta)
    if (ii % 100) == 0:
        print("==== The " + str(ii) + " iteration estimate is: ====")
        print(decode(theta))
    if np.exp(logpost(thetanew) - logpost(theta)) > np.random.uniform():
        theta = thetanew


==== The 0 iteration estimate is: ====
wjspvbfdnbdwvadhgbnqvj kahbdfadhbpxbkfabvdkanbj hjudkahbsgbua kvdwbsd lbqpwjujanbd bduuawavdkjp bkfdkbfdnbsdxxwahbaup pojnknbd hbkvdhavnbkfdkbijha j cbcdqbfdnbdwdvoahbkfpnabifpbidkufbjkbdnbdbnjc dwbpxbnkvannbj bkfabxj d ujdwbngnkaobpkfavnbfdmabqj  ahbjkbp bdbnavjanbpxbkauf judwbxdukpvnbnzufbdnbvjnj cbnfpvkkavobhaskbndwanbsgbkfabznbcpmav oa kbd hb aibupvqpvdkabkdrbqpwjujanbpkfavbodvlaknbkfdkbud bsabkdqqahbxpvbhpwwdvnbj uwzhj cbkfvpzcfbkfabnidqnbodvlakbd hbwjyzjhjkgbwj anbodj kdj ahbsgbcwpsdwbua kvdwbsd lnbdva kbgakbnfpij cbdbsjcbhpwwdvbnyzaata
==== The 100 iteration estimate is: ====
vsberazonaovrgohdanfrs tghaozgohaepatzgarotgnas hsiotghabdaig trovabo uafevsisgnao aoiigvgrotse atzotazonaboppvghagie eysntnao hatrohgrnatzotalshg s cacofazonaovoryghatzengalzealotizastaonaoansc ovaepantrgnnas atzgaps o isovandntgyaetzgrnazojgafs  ghastae aoangrsgnaepatgiz siovapoiternanmizaonarsns canzerttgryahgbtanovgnabdatzgamnacejgr yg tao ha glaierferotgatokafevsi

In [191]:
decode(theta)

'libor has already sprinted ahead of the rates indicated by central bank policies an acceleration that has baffled economists and traders that widening gap has alarmed those who watch it as a signal of stress in the financial system others have pinned it on a series of technical factors such as rising shortterm debt sales by the us government and new corporate taz policies other markets that can be tapped for dollars including through the swaps market and liquidity lines maintained by global central banks arent yet showing a big dollar squeexe'

There are 27 factorial different possible encodings. In a Markov chain with 10,000 iterations you check at most 

In [444]:
10./np.math.factorial(27)

9.183689863795546e-28

fraction of all encodings.

### Exercises for Extra Practice

** Exercise: ** Try other proposal distributions, such as randomly permuting 3 letters. Does this improve / worsen convergence?

** Exercise: ** Rerun this demo with a different coded message. How does you ability to recover the truth change with message length?

** Exercise: ** We computed bigram counts from "War and Peace." More generally one could compute [ngrams](https://en.wikipedia.org/wiki/N-gram). Try finding all 3 grams and calculating the conditional problability of any letter given the last two letters. So P(hello) = $P(h)P(e|h)P(l|he)P(l|el)P(o|ll)$.

** Exercise: ** Write a "random word generator." Here the input to the function is a character sequence length $n$ and the output is a random text sequence drawn from a 1,2,or 3 gram distribution. Random sequences from 3 gram distributions should produce character strings with many real words.

** Exercise: ** We are really more focused on the mode of the posterior than the full posterior distribution. Since the prior is flat, the mode of the posterior is equivalent to maximum likelihood estimation. Simulated tempering (Section 26.6 of Lange) is one method for finding the MLE / posterior mode for multimodal models. It shares many similarities with Metropolis. Implement simulated annealing for this problem. Does you algorithm converge more often to the correct solution than Metropolis?

Read more about this problem (and a lot of MCMC theory) in [The MCMC Revolution](https://pdfs.semanticscholar.org/4e62/58ebd07b548f77f8cd33caf72d811c2bd54c.pdf).